In [1]:
import sys
sys.path.append("D:/Python/siemens-rag-bot/siemens-energy/")

In [2]:
import os, re
import torch
import shutil
import random
import pandas as pd
from PIL import Image
from pathlib import Path
from itertools import islice
from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from collections import defaultdict
from diffusers import StableDiffusionPipeline
from huggingface_hub import hf_hub_download

import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
excel_path = '../dataset/captions.xlsx'  # change this
df = pd.read_excel(excel_path)
print(f"{df.shape=}")

bottle_df = df[df['Path'].str.contains('bottle')]
print(f"{bottle_df.shape=}")

df.shape=(917, 3)
bottle_df.shape=(63, 3)


D:\Python\siemens-rag-bot\env\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [4]:
bottle_df

,Path,object description,defect description
373,bottle/broken_large/000.png,"The image shows a top-down view of a bottle, l...",The bottom right corner has a large breakage d...
374,bottle/broken_large/001.png,"The image shows a top-down view of a bottle, l...","The lower half has a large breakage defect, sh..."
375,bottle/broken_large/002.png,"The image shows a top-down view of a bottle, l...","The upper half has a large breakage defect, sh..."
376,bottle/broken_large/003.png,"The image shows a top-down view of a bottle, l...","The left half has a large breakage defect, sha..."
377,bottle/broken_large/004.png,"The image shows a top-down view of a bottle, l...",The upper left corner has a small breakage def...
...,...,...,...
431,bottle/contamination/016.png,"The image shows a top-down view of a bottle, l...","Inside the bottle, there is a larger circular ..."
432,bottle/contamination/017.png,"The image shows a top-down view of a bottle, l...","Extending to the upper left inside the bottle,..."
433,bottle/contamination/018.png,"The image shows a top-down view of a bottle, l...",Extending to the upper right inside the bottle...
434,bottle/contamination/019.png,"The image shows a top-down view of a bottle, l...",The lower right corner has a large breakage de...


In [5]:
# ### !!! Note: This code is one time run only when copied the data from main data folder to dataset folder for respective class 

# path = '../dataset/bottle/image/'
# folders = os.listdir(path)
# print(folders)

# #for index, file in enumerate(files):
# #    os.rename(os.path.join(path, file), os.path.join(path, ''.join([str('raw_')+str(index), '.jpg'])))

# for i in folders:
#     new_path = path+i
#     files = os.listdir(new_path)
#     for index, file in enumerate(files):
#         fname = file.split(".")[0]
#         row = bottle_df.loc[bottle_df["Path"] == "bottle/" + i + "/" + file].values
#         if len(row)>0:
#             caption = row[0][1] + " " + row[0][2]
#             caption_path = os.path.join(new_path, ''.join([i+"-"+str(fname), '.txt']))
#             with open(caption_path, 'w') as f:
#                 f.write(caption)
#             os.rename(os.path.join(new_path, file), os.path.join(new_path, ''.join([i+"-"+str(fname), '.png'])))
#         else:
#             os.rename(os.path.join(new_path, file), os.path.join(new_path, ''.join([i+"-"+str(fname), '.png'])))

In [4]:
### !!! Note: This code is one time run only when copied the data from main data folder to dataset folder for respective class 

path = '../dataset/bottle/rbg_mask/'
folders = os.listdir(path)
print(folders)

#for index, file in enumerate(files):
#    os.rename(os.path.join(path, file), os.path.join(path, ''.join([str('raw_')+str(index), '.jpg'])))

for i in folders:
    new_path = path+i
    files = os.listdir(new_path)
    for index, file in enumerate(files):
        fname = file.split(".")[0]
        os.rename(os.path.join(new_path, file), os.path.join(new_path, ''.join([i+"-"+str(fname), '.png'])))

['broken_large', 'broken_small', 'contamination']


In [24]:
from scripts.dataset import BottleDataset

dataset = BottleDataset(root_dir="../dataset/bottle/image/", image_size=512)
loader = DataLoader(dataset, batch_size=4, shuffle=True)

sample = next(iter(loader))
print(sample['pixel_values'].shape)  # torch.Size([4, 3, 512, 512])
print(sample['text'])                # 4 captions

torch.Size([4, 3, 512, 512])
["The image shows a top-down view of a bottle, looking from the bottle's neck towards the bottom, with a white background. The left half has a large breakage defect, shaped like half a ring, in black color.", "The image shows a top-down view of a bottle, looking from the bottle's neck towards the bottom, with a white background. The left half has a large breakage defect, shaped like half a ring, in black color.", "The image shows a top-down view of a bottle, looking from the bottle's neck towards the bottom, with a white background. The upper right corner has a large breakage defect, shaped like a quarter of a ring, in black color.", "The image shows a top-down view of a bottle, looking from the bottle's neck towards the bottom, with a white background. Extending to the upper left inside the bottle, there is a large irregularly shaped black fabric contaminant."]


## Loading the model

In [25]:
model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, revision="fp16", use_auth_token=True).to("cuda")  # if private model

print("Model Loaded Successfully!")

model_index.json:   0%|          | 0.00/543 [00:00<?, ?B/s]

D:\Python\siemens-rag-bot\env\lib\site-packages\diffusers\pipelines\pipeline_loading_utils.py:285: FutureWarning: You are loading the variant fp16 from CompVis/stable-diffusion-v1-4 via `revision='fp16'`. This behavior is deprecated and will be removed in diffusers v1. One should use `variant='fp16'` instead. However, it appears that CompVis/stable-diffusion-v1-4 currently does not have the required variant filenames in the 'main' branch. 
 The Diffusers team and community would be very grateful if you could open an issue: https://github.com/huggingface/diffusers/issues/new with the title 'CompVis/stable-diffusion-v1-4 is missing fp16 files' so that the correct variant file can be added.
  warnings.warn(


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/246M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.63k [00:00<?, ?B/s]

...

diffusion_pytorch_model.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

Keyword arguments {'use_auth_token': True} are not expected by StableDiffusionPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch C:\Users\adity\.cache\huggingface\hub\models--CompVis--stable-diffusion-v1-4\snapshots\2880f2ca379f41b0226444936bb7a6766a227587\vae: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\adity\.cache\huggingface\hub\models--CompVis--stable-diffusion-v1-4\snapshots\2880f2ca379f41b0226444936bb7a6766a227587\vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch C:\Users\adity\.cache\huggingface\hub\models--CompVis--stable-diffusion-v1-4\snapshots\2880f2ca379f41b0226444936bb7a6766a227587\unet: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\adity\.cache\huggingface\hub\models--CompVis--stable-diffusion-v1-4\snapshots\2880f2ca379f41b0226444936bb7a6766a227587\unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
D:\Python\siemens-rag-bot\env\lib\site-packages\transforme

✅ Model Loaded Successfully!


In [27]:
from peft import get_peft_model, LoraConfig #, TaskType
from peft.utils.peft_types import TaskType

In [ ]:
lora_config = LoraConfig(
    r=4,
    lora_alpha=32,
    target_modules=["to_k", "to_q", "to_v", "to_out"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.
)
